# WITH PANDAS

In [2]:
import pandas as pd
from datetime import datetime


# Create a dataframe
data = {'finr': [361, 361, 361, 361],
        'woz_objectnr': [321, 321, 300, 300],
        'geldig_van_dt': ['2015-01-01', '2019-01-01', '2019-01-01', '2019-01-01'],
        'geldig_tot_dt': ['9999-12-31', '9999-12-31', '9999-12-31', '9999-12-31'],
        'Ind': ['E', 'G', 'G', 'E']}

df = pd.DataFrame(data)
df

,finr,woz_objectnr,geldig_van_dt,geldig_tot_dt,Ind
0,361,321,2015-01-01,9999-12-31,E
1,361,321,2019-01-01,9999-12-31,G
2,361,300,2019-01-01,9999-12-31,G
3,361,300,2019-01-01,9999-12-31,E


In [3]:
# Add jaar column according to the conditions

df['jaar'] = ""
for i in range(len(df)):
    if df['woz_objectnr'][i] == 321 and datetime.strptime(df['geldig_van_dt'][i], '%Y-%m-%d').year == 2015:
        df.at[i, 'jaar'] = [2018, 2019, 2020, 2021, 2022]
    elif df['woz_objectnr'][i] == 300 and datetime.strptime(df['geldig_van_dt'][i], '%Y-%m-%d').year == 2019:
        df.at[i, 'jaar'] = [2019, 2020, 2021, 2022]
df

,finr,woz_objectnr,geldig_van_dt,geldig_tot_dt,Ind,jaar
0,361,321,2015-01-01,9999-12-31,E,"[2018, 2019, 2020, 2021, 2022]"
1,361,321,2019-01-01,9999-12-31,G,
2,361,300,2019-01-01,9999-12-31,G,"[2019, 2020, 2021, 2022]"
3,361,300,2019-01-01,9999-12-31,E,"[2019, 2020, 2021, 2022]"


In [4]:
# Expload the dataframe by "jaar"
df_exploded = df.explode('jaar')
df_exploded

,finr,woz_objectnr,geldig_van_dt,geldig_tot_dt,Ind,jaar
0,361,321,2015-01-01,9999-12-31,E,2018
0,361,321,2015-01-01,9999-12-31,E,2019
0,361,321,2015-01-01,9999-12-31,E,2020
0,361,321,2015-01-01,9999-12-31,E,2021
0,361,321,2015-01-01,9999-12-31,E,2022
1,361,321,2019-01-01,9999-12-31,G,
2,361,300,2019-01-01,9999-12-31,G,2019
2,361,300,2019-01-01,9999-12-31,G,2020
2,361,300,2019-01-01,9999-12-31,G,2021
2,361,300,2019-01-01,9999-12-31,G,2022


# WITH POSTGRESQL

In [6]:
# Queries

# Create a table
query1="CREATE TABLE mytable (finr INT, woz_objectnr INT, geldig_van_dt DATE, geldig_tot_dt DATE, Ind VARCHAR(1));"

# Insert data
query2= "INSERT INTO mytable (finr, woz_objectnr, geldig_van_dt, geldig_tot_dt, Ind) VALUES (361, 321, '2015-01-01', '9999-12-31', 'E'),(361, 321, '2019-01-01', '9999-12-31', 'G'),(361, 300, '2019-01-01', '9999-12-31', 'G'),(361, 300, '2019-01-01', '9999-12-31', 'E');"

# Add "jaar" column
query3= "ALTER TABLE mytable ADD COLUMN jaar TEXT;"

# Update "jaar" column according to the conditions
query4 = """UPDATE mytable SET jaar = '2018,2019,2020,2021,2022' WHERE woz_objectnr = 321 AND EXTRACT(YEAR FROM geldig_van_dt) = 2015;
            UPDATE mytable SET jaar = '2019,2020,2021,2022' WHERE woz_objectnr = 300 AND EXTRACT(YEAR FROM geldig_van_dt) = 2019;"""

# Select all rows
query5= "SELECT *, UNNEST(string_to_array(jaar, ',')) AS years FROM mytable"

In [7]:
!pip install psycopg2-binary
import psycopg2

# PostgreSQL veritabanı bağlantısı oluşturma
conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="deneme",
    user="postgres",
    password="1234"
)

# Cursor oluştur
cur = conn.cursor()

# SQL sorgusu
cur.execute(query1)
cur.execute(query2)
cur.execute(query3)
cur.execute(query4)
df_exploded = pd.read_sql_query(query5,conn)

# Bağlantıyı kapatma
cur.close()
conn.close()

df_exploded

,finr,woz_objectnr,geldig_van_dt,geldig_tot_dt,ind,jaar,years
0,361,321,2015-01-01,9999-12-31,E,"2018,2019,2020,2021,2022",2018
1,361,321,2015-01-01,9999-12-31,E,"2018,2019,2020,2021,2022",2019
2,361,321,2015-01-01,9999-12-31,E,"2018,2019,2020,2021,2022",2020
3,361,321,2015-01-01,9999-12-31,E,"2018,2019,2020,2021,2022",2021
4,361,321,2015-01-01,9999-12-31,E,"2018,2019,2020,2021,2022",2022
5,361,300,2019-01-01,9999-12-31,G,"2019,2020,2021,2022",2019
6,361,300,2019-01-01,9999-12-31,G,"2019,2020,2021,2022",2020
7,361,300,2019-01-01,9999-12-31,G,"2019,2020,2021,2022",2021
8,361,300,2019-01-01,9999-12-31,G,"2019,2020,2021,2022",2022
9,361,300,2019-01-01,9999-12-31,E,"2019,2020,2021,2022",2019
